In [21]:
import numpy as np

def tweak1(mat):
    matrix_C = mat.copy()
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            matrix_C[i, j] = mat[i, (j+i) % mat.shape[1]]
    return matrix_C

def tweak2(mat):
    matrix_C = mat.copy()
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            matrix_C[i, j] = mat[(j+i) % mat.shape[0], j]
    return matrix_C

floor = -1.0
ceil = 1.0
A = []
for i in range(4):
    data = (ceil-floor) * np.random.rand(128*128) + floor
    A.append(data.reshape(128, 128))

sigma_A = []
for i in range(4):
    sigma_A.append(tweak1(A[i]))

sigma_A_1 = []
sigma_A_2 = []
for i in range(4):
    sigma_A_1.append(sigma_A[i][:, :64])
    sigma_A_2.append(sigma_A[i][:, 64:])
sigma_A_1_write = np.concatenate(sigma_A_1, axis=1)
sigma_A_2_write = np.concatenate(sigma_A_2, axis=1)
with open(f"data/sigma_A_1.txt", "w") as f:
    f.write(" ".join(map(str, sigma_A_1_write.flatten())))
with open(f"data/sigma_A_2.txt", "w") as f:
    f.write(" ".join(map(str, sigma_A_2_write.flatten())))


V = []
for i in range(4):
    data = (ceil-floor) * np.random.rand(128*64) + floor
    V.append(data.reshape(128, 64))
res_V = np.concatenate(V, axis=1)
tau_V = []
for i in range(4):
    tau_V.append(tweak2(V[i]))
tau_V_write = np.concatenate(tau_V, axis=1)
with open(f"data/tau_V.txt", "w") as f:
    f.write(" ".join(map(str, tau_V_write.flatten())))
with open(f"data/V.txt", "w") as f:
    f.write(" ".join(map(str, res_V.flatten())))

mul_res = []
for i in range(4):
    mul_res.append(A[i] @ V[i])

res_write = np.concatenate(mul_res, axis=1)
# with open(f"mul_V_res.txt", "w") as f:
#     f.write(" ".join(map(str, res_write.flatten())))


mask_L = [None] * 64
for sed in range(1,64):
    mask_L[sed] = np.zeros(32768)
    for i in range(128):
        for j in range(64):
            for k in range(4):
                if (j < 64-sed):
                    mask_L[sed][i*256 + k*64 + j] = 1.0

mask_R = [None] * 64
for sed in range(1,64):
    mask_R[sed] = np.zeros(32768)
    for i in range(128):
        for j in range(64):
            for k in range(4):
                if (j > 64-sed-1):
                    mask_R[sed][i*256 + k*64 + j] = 1.0

O1 = sigma_A_1_write.copy().reshape(32768)
O2 = sigma_A_2_write.copy().reshape(32768)
rot_V = tau_V_write.copy().reshape(32768)
ct0 = O1 * rot_V
for i in range(1,64):
    rot_V = np.roll(rot_V, -256)
    O1 = np.roll(O1, -1)
    O1_mask = O1 * mask_L[i]
    if (i == 1):
        O2 = np.roll(O2, 64-1)
    else:
        O2 = np.roll(O2, -1)
    O2_mask = O2 * mask_R[i]
    mul1 = O1_mask + O2_mask
    ct0 += mul1 * rot_V


O1 = sigma_A_2_write.copy().reshape(32768)
O2 = sigma_A_1_write.copy().reshape(32768)
rot_V = np.roll(rot_V, -256)
ct0 += O1 * rot_V
for i in range(1,64):
    rot_V = np.roll(rot_V, -256)
    O1 = np.roll(O1, -1)
    O1_mask = O1 * mask_L[i]
    if (i == 1):
        O2 = np.roll(O2, 64-1)
    else:
        O2 = np.roll(O2, -1)
    O2_mask = O2 * mask_R[i]
    mul1 = O1_mask + O2_mask
    ct0 += mul1 * rot_V

res_write = res_write.reshape(32768)
error = 0
for i in range(32768):
    error = max(error, abs(res_write[i]-ct0[i]))
print(error)

with open(f"data/mul_V_res.txt", "w") as f:
    f.write(" ".join(map(str, ct0.flatten())))

1.509903313490213e-14


In [ ]:
# bsgs version
# 因为密钥太多放弃了
import numpy as np

def tweak1(mat):
    matrix_C = mat.copy()
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            matrix_C[i, j] = mat[i, (j+i) % mat.shape[1]]
    return matrix_C

def tweak2(mat):
    matrix_C = mat.copy()
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            matrix_C[i, j] = mat[(j+i) % mat.shape[0], j]
    return matrix_C

floor = -1.0
ceil = 1.0
A = []
for i in range(4):
    data = (ceil-floor) * np.random.rand(128*128) + floor
    A.append(data.reshape(128, 128))

sigma_A = []
for i in range(4):
    sigma_A.append(tweak1(A[i]))

sigma_A_1 = []
sigma_A_2 = []
for i in range(4):
    sigma_A_1.append(sigma_A[i][:, :64])
    sigma_A_2.append(sigma_A[i][:, 64:])
sigma_A_1_write = np.concatenate(sigma_A_1, axis=1)
sigma_A_2_write = np.concatenate(sigma_A_2, axis=1)
with open(f"data/sigma_A_1.txt", "w") as f:
    f.write(" ".join(map(str, sigma_A_1_write.flatten())))
with open(f"data/sigma_A_2.txt", "w") as f:
    f.write(" ".join(map(str, sigma_A_2_write.flatten())))


V = []
for i in range(4):
    data = (ceil-floor) * np.random.rand(128*64) + floor
    V.append(data.reshape(128, 64))
res_V = np.concatenate(V, axis=1)
tau_V = []
for i in range(4):
    tau_V.append(tweak2(V[i]))
tau_V_write = np.concatenate(tau_V, axis=1)
with open(f"data/tau_V.txt", "w") as f:
    f.write(" ".join(map(str, tau_V_write.flatten())))
with open(f"data/V.txt", "w") as f:
    f.write(" ".join(map(str, res_V.flatten())))

mul_res = []
for i in range(4):
    mul_res.append(A[i] @ V[i])

res_write = np.concatenate(mul_res, axis=1)
# with open(f"mul_V_res.txt", "w") as f:
#     f.write(" ".join(map(str, res_write.flatten())))

mask_R = [None] * 64
mask_R[0] = np.ones(32768)
for sed in range(1,64):
    mask_R[sed] = np.zeros(32768)
    for i in range(128):
        for j in range(64):
            for k in range(4):
                if (j > sed-1):
                    mask_R[sed][i*256 + k*64 + j] = 1.0
print(f"mask_R[1]{mask_R[1]}")

mask_L = [None] * 64
mask_L[0] = np.zeros(32768)
for sed in range(1,64):
    mask_L[sed] = np.zeros(32768)
    for i in range(128):
        for j in range(64):
            for k in range(4):
                if (j < sed):
                    mask_L[sed][i*256 + k*64 + j] = 1.0
print(f"mask_L[1]{mask_L[1]}")


O1 = sigma_A_1_write.copy().reshape(32768)
O2 = sigma_A_2_write.copy().reshape(32768)
rot_V = tau_V_write.copy().reshape(32768)
O1_p = np.roll(O1, 64)
O2_p = np.roll(O2, 64)
gs = 16
bs = 8
bs_res = []

for i in range(bs):
    bs_res.append(np.roll(rot_V, -i*256))

ct0 = np.zeros(32768)
rot_lis = []
for i in range(gs):
    ct_temp = np.zeros(32768)
    for j in range(bs):
        offset = i*bs+j
        rot_lis.append((bs*i+j-bs*i*256)%32768)
        if (offset < 64):
            ct_temp += bs_res[j] * np.roll(O1*mask_R[offset] + O2_p*mask_L[offset], -(offset)+i*bs*256)
        else:
            offset -= 64
            ct_temp += bs_res[j] * np.roll(O2*mask_R[offset] + O1_p*mask_L[offset], -(offset)+i*bs*256)
    ct0 += np.roll(ct_temp,-bs*256*i)

print(rot_lis)

res_write = res_write.reshape(32768)
error = 0
for i in range(32768):
    error = max(error, abs(res_write[i]-ct0[i]))
print(error)

with open(f"data/mul_V_res.txt", "w") as f:
    f.write(" ".join(map(str, ct0.flatten())))

mask_R[1][0. 1. 1. ... 1. 1. 1.]
mask_L[1][1. 0. 0. ... 0. 0. 0.]
[0, 1, 2, 3, 4, 5, 6, 7, 30728, 30729, 30730, 30731, 30732, 30733, 30734, 30735, 28688, 28689, 28690, 28691, 28692, 28693, 28694, 28695, 26648, 26649, 26650, 26651, 26652, 26653, 26654, 26655, 24608, 24609, 24610, 24611, 24612, 24613, 24614, 24615, 22568, 22569, 22570, 22571, 22572, 22573, 22574, 22575, 20528, 20529, 20530, 20531, 20532, 20533, 20534, 20535, 18488, 18489, 18490, 18491, 18492, 18493, 18494, 18495, 16448, 16449, 16450, 16451, 16452, 16453, 16454, 16455, 14408, 14409, 14410, 14411, 14412, 14413, 14414, 14415, 12368, 12369, 12370, 12371, 12372, 12373, 12374, 12375, 10328, 10329, 10330, 10331, 10332, 10333, 10334, 10335, 8288, 8289, 8290, 8291, 8292, 8293, 8294, 8295, 6248, 6249, 6250, 6251, 6252, 6253, 6254, 6255, 4208, 4209, 4210, 4211, 4212, 4213, 4214, 4215, 2168, 2169, 2170, 2171, 2172, 2173, 2174, 2175]
1.5987211554602254e-14


In [28]:
# bsgs version
# 旋转次数更多，但需要的密钥更少
import numpy as np

def tweak1(mat):
    matrix_C = mat.copy()
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            matrix_C[i, j] = mat[i, (j+i) % mat.shape[1]]
    return matrix_C

def tweak2(mat):
    matrix_C = mat.copy()
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            matrix_C[i, j] = mat[(j+i) % mat.shape[0], j]
    return matrix_C

floor = -1.0
ceil = 1.0
A = []
for i in range(4):
    data = (ceil-floor) * np.random.rand(128*128) + floor
    A.append(data.reshape(128, 128))

sigma_A = []
for i in range(4):
    sigma_A.append(tweak1(A[i]))

sigma_A_1 = []
sigma_A_2 = []
for i in range(4):
    sigma_A_1.append(sigma_A[i][:, :64])
    sigma_A_2.append(sigma_A[i][:, 64:])
sigma_A_1_write = np.concatenate(sigma_A_1, axis=1)
sigma_A_2_write = np.concatenate(sigma_A_2, axis=1)
with open(f"data/sigma_A_1.txt", "w") as f:
    f.write(" ".join(map(str, sigma_A_1_write.flatten())))
with open(f"data/sigma_A_2.txt", "w") as f:
    f.write(" ".join(map(str, sigma_A_2_write.flatten())))


V = []
for i in range(4):
    data = (ceil-floor) * np.random.rand(128*64) + floor
    V.append(data.reshape(128, 64))
res_V = np.concatenate(V, axis=1)
tau_V = []
for i in range(4):
    tau_V.append(tweak2(V[i]))
tau_V_write = np.concatenate(tau_V, axis=1)
with open(f"data/tau_V.txt", "w") as f:
    f.write(" ".join(map(str, tau_V_write.flatten())))
with open(f"data/V.txt", "w") as f:
    f.write(" ".join(map(str, res_V.flatten())))

mul_res = []
for i in range(4):
    mul_res.append(A[i] @ V[i])

res_write = np.concatenate(mul_res, axis=1)
# with open(f"mul_V_res.txt", "w") as f:
#     f.write(" ".join(map(str, res_write.flatten())))

mask_R = [None] * 64
mask_R[0] = np.ones(32768)
for sed in range(1,64):
    mask_R[sed] = np.zeros(32768)
    for i in range(128):
        for j in range(64):
            for k in range(4):
                if (j > sed-1):
                    mask_R[sed][i*256 + k*64 + j] = 1.0
print(f"mask_R[1]{mask_R[1]}")

mask_L = [None] * 64
mask_L[0] = np.zeros(32768)
for sed in range(1,64):
    mask_L[sed] = np.zeros(32768)
    for i in range(128):
        for j in range(64):
            for k in range(4):
                if (j < sed):
                    mask_L[sed][i*256 + k*64 + j] = 1.0
print(f"mask_L[1]{mask_L[1]}")


O1 = sigma_A_1_write.copy().reshape(32768)
O2 = sigma_A_2_write.copy().reshape(32768)
rot_V = tau_V_write.copy().reshape(32768)
O1_p = np.roll(O1, 64)
O2_p = np.roll(O2, 64)
gs = 16
bs = 8
gs_res = []

# 大步需要gs次旋转，即16次
for i in range(gs):
    gs_res.append(np.roll(rot_V, -i*bs*256))

ct0 = np.zeros(32768)
for i in range(bs):
    # 需要2*bs-1次旋转，即18次
    ct_temp = np.zeros(32768)
    ct1 = np.roll(O1, 0+i*256)
    ct2 = np.roll(O2, 64+i*256)
    ct3 = np.roll(O2, 0+i*256)
    ct4 = np.roll(O1, 64+i*256)
    
    for j in range(gs):
        offset = i+j*bs
        # gs*b*2次旋转，即16*8*2次
        if (offset < 64):
            # ct_temp += gs_res[j] * (np.roll(ct1*mask_R[offset], -offset) + np.roll(ct2*mask_L[offset], -offset))
            # 在fhe里，先分别rot再mask
            ct_temp += gs_res[j] * np.roll(ct1*mask_R[offset]+ct2*mask_L[offset], -offset)
        else:
            offset -= 64
            ct_temp += gs_res[j] * (np.roll(ct3*mask_R[offset], -offset) + np.roll(ct4*mask_L[offset], -offset))
    # 需要bs次旋转，即8次
    ct0 += np.roll(ct_temp,-256*i)
# 共需要gs + 2*bs-1 + gs*bs*2 + bs 次旋转

res_write = res_write.reshape(32768)
error = 0
right = 0
for i in range(32768):
    if abs(res_write[i]-ct0[i]) < 1e-5:
        right += 1
    error = max(error, abs(res_write[i]-ct0[i]))
print(error)
print(right)


with open(f"data/mul_V_res.txt", "w") as f:
    f.write(" ".join(map(str, ct0.flatten())))

mask_R[1][0. 1. 1. ... 1. 1. 1.]
mask_L[1][1. 0. 0. ... 0. 0. 0.]
1.3322676295501878e-14
32768


In [1]:
# bsgs version 按fhe逻辑修改后的
# 旋转次数更多，但需要的密钥更少
import numpy as np

def tweak1(mat):
    matrix_C = mat.copy()
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            matrix_C[i, j] = mat[i, (j+i) % mat.shape[1]]
    return matrix_C

def tweak2(mat):
    matrix_C = mat.copy()
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            matrix_C[i, j] = mat[(j+i) % mat.shape[0], j]
    return matrix_C

floor = -1.0
ceil = 1.0
A = []
for i in range(4):
    data = (ceil-floor) * np.random.rand(128*128) + floor
    A.append(data.reshape(128, 128))

sigma_A = []
for i in range(4):
    sigma_A.append(tweak1(A[i]))

sigma_A_1 = []
sigma_A_2 = []
for i in range(4):
    sigma_A_1.append(sigma_A[i][:, :64])
    sigma_A_2.append(sigma_A[i][:, 64:])
sigma_A_1_write = np.concatenate(sigma_A_1, axis=1)
sigma_A_2_write = np.concatenate(sigma_A_2, axis=1)
with open(f"data/sigma_A_1.txt", "w") as f:
    f.write(" ".join(map(str, sigma_A_1_write.flatten())))
with open(f"data/sigma_A_2.txt", "w") as f:
    f.write(" ".join(map(str, sigma_A_2_write.flatten())))


V = []
for i in range(4):
    data = (ceil-floor) * np.random.rand(128*64) + floor
    V.append(data.reshape(128, 64))
res_V = np.concatenate(V, axis=1)
tau_V = []
for i in range(4):
    tau_V.append(tweak2(V[i]))
tau_V_write = np.concatenate(tau_V, axis=1)
with open(f"data/tau_V.txt", "w") as f:
    f.write(" ".join(map(str, tau_V_write.flatten())))
with open(f"data/V.txt", "w") as f:
    f.write(" ".join(map(str, res_V.flatten())))

mul_res = []
for i in range(4):
    mul_res.append(A[i] @ V[i])

res_write = np.concatenate(mul_res, axis=1)
# with open(f"mul_V_res.txt", "w") as f:
#     f.write(" ".join(map(str, res_write.flatten())))

mask_R = [None] * 64
mask_R[0] = np.zeros(32768)
for sed in range(1,64):
    mask_R[sed] = np.zeros(32768)
    for i in range(128):
        for j in range(64):
            for k in range(4):
                if (j > 64-sed-1):
                    mask_R[sed][i*256 + k*64 + j] = 1.0
print(f"mask_R[1]{mask_R[1]}")

mask_L = [None] * 64
mask_L[0] = np.ones(32768)
for sed in range(1,64):
    mask_L[sed] = np.zeros(32768)
    for i in range(128):
        for j in range(64):
            for k in range(4):
                if (j < 64 - sed):
                    mask_L[sed][i*256 + k*64 + j] = 1.0
print(f"mask_L[1]{mask_L[1]}")

O1 = sigma_A_1_write.copy().reshape(32768)
O2 = sigma_A_2_write.copy().reshape(32768)
rot_V = tau_V_write.copy().reshape(32768)
O1_p = np.roll(O1, 64)
O2_p = np.roll(O2, 64)
gs = 16
bs = 8
gs_res = []

# 大步需要gs-1次旋转，即15次
for i in range(gs):
    gs_res.append(np.roll(rot_V, -i*bs*256))

ct0 = np.zeros(32768)
splited_piece = 2

for i in range(bs):
    # 需要2*(bs-1)次旋转，即14次
    ct_temp = np.zeros(32768)
    ct1 = np.roll(O1, 0+i*(256-1))
    ct3 = np.roll(O2, 0+i*(256-1))

    for j in range(gs//splited_piece):
        # 2*bs*gs-2次旋转，即254次
        # bs*gs次乘法， 即128次
        offset = i+j*bs
        if (j!=0):
            ct1 = np.roll(ct1, -bs)
            ct3 = np.roll(ct3, -bs)
        ct4 = np.roll(ct1, 64)
        ct2 = np.roll(ct3, 64)
        ct_temp += gs_res[j] * (ct1*mask_L[offset]+ct2*mask_R[offset])
        ct_temp += gs_res[j+8] * (ct3*mask_L[offset]+ct4*mask_R[offset])
    # 需要bs-1次旋转，即7次
    ct0 += np.roll(ct_temp,-256*i)
# 共需要gs-1 + 2*(bs-1) + 2*bs*gs-2 + bs-1 次旋转,即290次

res_write = res_write.reshape(32768)
error = 0
right = 0
for i in range(32768):
    if abs(res_write[i]-ct0[i]) < 1e-5:
        right += 1
    error = max(error, abs(res_write[i]-ct0[i]))
print(error)
print(f"正确数量为{right}")
print(res_write[0])
print(ct0[0])

with open(f"data/mul_V_res.txt", "w") as f:
    f.write(" ".join(map(str, ct0.flatten())))

mask_R[1][0. 0. 0. ... 0. 0. 1.]
mask_L[1][1. 1. 1. ... 1. 1. 0.]
1.2434497875801753e-14
正确数量为32768
0.2911943143061144
0.29119431430611475


In [22]:
print(gs_res[15][:8])

[-0.97136616  0.73086126 -0.84045141 -0.33000526  0.18059136 -0.85509948
  0.6562844  -0.09161788]
